In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from datasets import Dataset
import torch
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from transformers import (
    BertTokenizerFast,
    BertForSequenceClassification,
    TrainingArguments,
    Trainer,
)
import evaluate
import wandb

In [9]:
emotion_list = [
    "admiration", "amusement", "anger", "annoyance", "approval", "caring", "confusion",
    "curiosity", "desire", "disappointment", "disapproval", "disgust", "embarrassment",
    "excitement", "fear", "gratitude", "grief", "joy", "love", "nervousness", "optimism",
    "pride", "realization", "relief", "remorse", "sadness", "surprise", "neutral"
]

ekman_mapping = {
    "anger": ["anger", "annoyance", "disapproval"],
    "disgust": ["disgust"],
    "fear": ["fear", "nervousness", "confusion"],
    "joy": ["admiration", "amusement", "approval", "caring", "desire", "excitement",
            "gratitude", "joy", "love", "optimism", "pride", "relief"],
    "sadness": ["sadness", "disappointment", "embarrassment", "grief", "remorse"],
    "surprise": ["surprise", "realization", "curiosity"],
    "neutral": ["neutral"]
}
ekman_to_index = {k: i for i, k in enumerate(ekman_mapping)}

def map_labels(raw_labels):
    new_ids = []
    for lid in raw_labels.split(','):
        emo = emotion_list[int(lid)]
        mapped = [ekman_to_index[k] for k, v in ekman_mapping.items() if emo in v]
        new_ids.extend(mapped)
    return list(set(new_ids))

def load_and_process(file):
    df = pd.read_csv(file, sep='\t', header=None, names=['text', 'labels', 'id'])
    df.drop(columns='id', inplace=True)
    df['labels'] = df['labels'].apply(map_labels)
    return df

train_df = load_and_process("train.tsv")
dev_df = load_and_process("dev.tsv")
test_df = load_and_process("test.tsv")

mlb = MultiLabelBinarizer(classes=list(range(7)))
train_df['labels'] = mlb.fit_transform(train_df['labels']).tolist()
dev_df['labels'] = mlb.transform(dev_df['labels']).tolist()
test_df['labels'] = mlb.transform(test_df['labels']).tolist()

In [9]:
train_ds = Dataset.from_pandas(train_df)
dev_ds = Dataset.from_pandas(dev_df)
test_ds = Dataset.from_pandas(test_df)

In [4]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

train_ds = train_ds.map(tokenize, batched=True)
dev_ds = dev_ds.map(tokenize, batched=True)
test_ds = test_ds.map(tokenize, batched=True)

c:\DAT301m\.venv\Lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|██████████| 5427/5427 [00:00<00:00, 14385.31 examples/s]


In [5]:
def convert_labels(example):
    example["labels"] = [float(x) for x in example["labels"]]
    return example

train_ds = train_ds.map(convert_labels)
dev_ds = dev_ds.map(convert_labels)
test_ds = test_ds.map(convert_labels)

train_ds.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
dev_ds.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_ds.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

Map: 100%|██████████| 5427/5427 [00:00<00:00, 15323.86 examples/s]


In [6]:
from datasets import Dataset
import pandas as pd

def df_to_dataset(df):
    return Dataset.from_dict({
        "text": df["text"].tolist(),
        "labels": df["labels"].apply(lambda x: [float(i) for i in x]).tolist()
    })

train_ds = df_to_dataset(train_df)
dev_ds = df_to_dataset(dev_df)
test_ds = df_to_dataset(test_df)


In [7]:
train_ds = train_ds.map(tokenize, batched=True)
dev_ds = dev_ds.map(tokenize, batched=True)
test_ds = test_ds.map(tokenize, batched=True)

train_ds.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
dev_ds.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_ds.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


Map: 100%|██████████| 5427/5427 [00:00<00:00, 14473.67 examples/s]


In [8]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=7,
    problem_type="multi_label_classification"
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
f1 = evaluate.load("f1")
accuracy = evaluate.load("accuracy")


def compute_metrics(p):
    logits, labels = p.predictions, p.label_ids
    probs = 1 / (1 + np.exp(-logits))  # sigmoid

    # Dự đoán: nếu xác suất > 0.5 thì là nhãn 1
    preds = (probs >= 0.5).astype(int)

    return {
        "f1_macro": f1_score(labels, preds, average="macro", zero_division=0),
        "precision_macro": precision_score(labels, preds, average="macro", zero_division=0),
        "recall_macro": recall_score(labels, preds, average="macro", zero_division=0),
        "accuracy": accuracy_score(labels, preds),
    }


In [10]:
training_args = TrainingArguments(
    output_dir="./bert-goemotions",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    logging_steps=50,
    report_to="wandb",
    run_name="bert-goemotions-ekman"
)

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=dev_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [12]:
trainer.train()

wandb: Currently logged in as: daiphat2312 (daiphat2312-fpt-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


  0%|          | 51/43412 [00:11<2:26:17,  4.94it/s]

{'loss': 0.4918, 'grad_norm': 1.6902152299880981, 'learning_rate': 1.9976964894499217e-05, 'epoch': 0.0}


  0%|          | 101/43412 [00:21<2:28:48,  4.85it/s]

{'loss': 0.3991, 'grad_norm': 2.3046875, 'learning_rate': 1.9953929788998436e-05, 'epoch': 0.01}


...

                                                       
100%|██████████| 43412/43412 [3:09:58<00:00,  5.33it/s]

{'eval_loss': 0.3436893820762634, 'eval_f1_macro': 0.5809349359556722, 'eval_precision_macro': 0.6001711797057409, 'eval_recall_macro': 0.5642203499394266, 'eval_accuracy': 0.599705123479543, 'eval_runtime': 89.29, 'eval_samples_per_second': 60.768, 'eval_steps_per_second': 7.604, 'epoch': 4.0}


100%|██████████| 43412/43412 [3:10:01<00:00,  3.81it/s]

{'train_runtime': 11404.4174, 'train_samples_per_second': 15.226, 'train_steps_per_second': 3.807, 'train_loss': 0.1715104971395092, 'epoch': 4.0}


TrainOutput(global_step=43412, training_loss=0.1715104971395092, metrics={'train_runtime': 11404.4174, 'train_samples_per_second': 15.226, 'train_steps_per_second': 3.807, 'total_flos': 1.1422163665152e+16, 'train_loss': 0.1715104971395092, 'epoch': 4.0})

In [13]:
trainer.save_model("bert-goemotions-final")


In [14]:
tokenizer.save_pretrained("bert-goemotions-final")


('bert-goemotions-final\\tokenizer_config.json',
 'bert-goemotions-final\\special_tokens_map.json',
 'bert-goemotions-final\\vocab.txt',
 'bert-goemotions-final\\added_tokens.json',
 'bert-goemotions-final\\tokenizer.json')

In [15]:
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained("bert-goemotions-final")
model = BertForSequenceClassification.from_pretrained(
    "bert-goemotions-final", 
    problem_type="multi_label_classification"
)


In [ ]:
trainer.evaluate(eval_dataset=test_ds)  


100%|██████████| 679/679 [01:28<00:00,  7.67it/s]


{'eval_loss': 0.2316652089357376,
 'eval_f1_macro': 0.5919607132686104,
 'eval_precision_macro': 0.6597682519226583,
 'eval_recall_macro': 0.5403244714443878,
 'eval_accuracy': 0.6073337018610651,
 'eval_runtime': 88.7951,
 'eval_samples_per_second': 61.118,
 'eval_steps_per_second': 7.647,
 'epoch': 4.0}

In [17]:
pred_output = trainer.predict(test_ds)


100%|██████████| 679/679 [01:28<00:00,  7.66it/s]


In [18]:
pred_output.predictions  # logits (float32)
pred_output.label_ids    # ground truth (multi-hot)


array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [20]:
text = "I hate eating salad."
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128).to(model.device)
with torch.no_grad():
    outputs = model(**inputs)
    probs = torch.sigmoid(outputs.logits).cpu().numpy()[0]
    labels = [i for i, p in enumerate(probs) if p >= 0.5]

ekman_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
predicted = [ekman_labels[i] for i in labels]
print("Predicted emotions:", predicted)


Predicted emotions: ['anger']


In [5]:
from transformers import BertTokenizer, BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("bert-goemotions-final")
tokenizer = BertTokenizer.from_pretrained("bert-goemotions-final")
model.eval()


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [11]:
def map_labels(labels):
    new_id_label = []
    for label_id in labels:  # labels là list các chỉ số dạng string hoặc int
        label = emotion_list[int(label_id)]  # chuyển chỉ số sang tên cảm xúc
        new_label = [ekman_to_index[k] for k, v in ekman_mapping.items() if label in v]
        new_id_label.extend(new_label)
    return list(set(new_id_label))  # loại trùng
test_df['new_labels'] = test_df['labels'].apply(map_labels)


In [12]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(classes=list(range(7)))
y_true = mlb.fit_transform(test_df['new_labels'])  # (N, 7)


In [13]:
test_encodings = tokenizer(
    test_df["text"].tolist(),
    padding=True,
    truncation=True,
    return_tensors="pt"
)


In [14]:
import torch
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(
    test_encodings["input_ids"],
    test_encodings["attention_mask"],
    torch.tensor(y_true, dtype=torch.float)
)
loader = DataLoader(dataset, batch_size=16)

y_preds = []
y_labels = []

with torch.no_grad():
    for batch in loader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = (torch.sigmoid(logits) > 0.5).int()

        y_preds.append(preds)
        y_labels.append(labels.int())

y_preds = torch.cat(y_preds).numpy()
y_true = torch.cat(y_labels).numpy()


In [15]:
from sklearn.metrics import hamming_loss, accuracy_score

hamming = hamming_loss(y_true, y_preds)
subset_acc = accuracy_score(y_true, y_preds)

print(f"Hamming Loss: {hamming:.4f}")
print(f"Subset Accuracy: {subset_acc:.4f}")


Hamming Loss: 0.1704
Subset Accuracy: 0.3586
